In [1]:
import pandas as pd
import numpy as np
import cvxpy as cp
from scipy import sparse

import time 

import multiprocessing
from itertools import repeat


import matplotlib.pyplot as plt
%matplotlib inline


from pyomo.environ import *

### This notebook solves problem (11) to identify the EV charging demand that can be supported in the San Francisco Bay area
- $\omega$ represents the percentage of charging demand that can be supported (shape: N_e_total).
- We totally have 19302 trip chains. Due to the memory limitation, we divide these trip chains into 22 groups with 1000 chains in each.
- It calculates the value of $\omega$ under different scales of EV hosting capacity, charger availability, and EV penetration

In [2]:
q = sparse.load_npz('Parameters/Bay/pubCharger/q_rec.npz').toarray()[0]
demand_cum = sparse.load_npz(f"Parameters/Bay/pubCharger/demand_cum.npz").toarray()
Location = sparse.load_npz(f"Parameters/Bay/pubCharger/Location.npz").toarray()
N_e_total = len(q)
N_n = 9
N_t = 96
E_init = 0.5
E_terminal = 0.5
L = np.zeros((N_t, N_t))
for i in range(N_t):
    for j in range(N_t):
        if i >= j:
            L[i, j] = 1

N_e = 1000
groupNum = N_e_total // N_e
N_e_last = N_e_total - N_e * groupNum
groupShare_list = np.zeros(groupNum+1)
for g in range(groupNum+1):
    groupShare_list[g] = q[g*N_e:(g+1)*N_e].sum()/q.sum()


index_to_county = {
    0: 'San Francisco',
    1: 'San Mateo',
    2: 'Santa Clara',
    3: 'Alameda',
    4: 'Contra Costa',
    5: 'Solano',
    6: 'Napa',
    7: 'Sonoma',
    8: 'Marin'
}

In [4]:
boolIndex = {}
for countyInd in range(9):
    boolIndex[countyInd] = np.zeros((N_e_total, N_t))
    for t in range(N_t):
        boolIndex[countyInd][:, t] = Location[:, t] == countyInd
    

In [4]:
'''problem for each group'''

model = ConcreteModel()

# Variables

model.y_e = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0)
model.E_e_omega = Var(range(N_e), range(N_t), domain=NonNegativeReals, initialize=0) # stored energy
model.omega = Var(range(N_e), domain=NonNegativeReals, bounds=(0, 1), initialize=1)

model.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model.x_e_UB = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # p_max
model.demand_cum = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # demand
model.q_e = Param(range(N_e), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model.coe_county0 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True) # pene * q_e * boolIndex[countyInd]
model.coe_county1 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county2 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county3 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county4 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county5 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county6 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county7 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)
model.coe_county8 = Param(range(N_e), range(N_t), domain=NonNegativeReals, mutable=True)



# Constraints for individual trip chain
model.c1 = ConstraintList()
for e in range(N_e):
    for t in range(N_t):
        model.c1.add(expr=model.y_e[e, t] <= model.omega[e] * model.x_e_UB[e, t])
        model.c1.add(expr=E_init * model.omega[e] - model.demand_cum[e, t] * model.omega[e] + 
                     sum(L[t, tt] * model.y_e[e, tt] * 0.25 for tt in range(N_t)) == model.E_e_omega[e, t])
        model.c1.add(expr=model.E_e_omega[e, t] >= 0)
        model.c1.add(expr=model.E_e_omega[e, t] <= model.omega[e])
    model.c1.add(expr=model.E_e_omega[e, N_t-1] >= model.omega[e] * E_terminal)

# Constraints for aggregate charging power capacity
model.c2 = ConstraintList()
for t in range(N_t):
    model.c2.add(sum(model.coe_county0[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[0])
    model.c2.add(sum(model.coe_county1[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[1])
    model.c2.add(sum(model.coe_county2[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[2])
    model.c2.add(sum(model.coe_county3[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[3])
    model.c2.add(sum(model.coe_county4[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[4])
    model.c2.add(sum(model.coe_county5[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[5])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[6])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[7])
    model.c2.add(sum(model.coe_county6[e, t] * model.y_e[e, t] for e in range(N_e)) <= model.ell_capacity[8])

# Objective
model.obj = Objective(expr=sum(model.q_e[e] * model.omega[e] for e in range(N_e)), sense=maximize)

In [5]:
'''problem for each group'''

model_last = ConcreteModel()

# Variables

model_last.y_e = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0)
model_last.E_e_omega = Var(range(N_e_last), range(N_t), domain=NonNegativeReals, initialize=0) # stored energy
model_last.omega = Var(range(N_e_last), domain=NonNegativeReals, bounds=(0, 1), initialize=1)

model_last.ell_capacity = Param(range(N_n), domain=NonNegativeReals, mutable=True)
model_last.x_e_UB = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # p_max
model_last.demand_cum = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # demand
model_last.q_e = Param(range(N_e_last), domain=NonNegativeReals, mutable=True)

# model_last.pene_para * model_last.q[e] * model_last.omega_para[e] * boolIndex[countyInd][e, t]
model_last.coe_county0 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True) # pene * q_e * boolIndex[countyInd]
model_last.coe_county1 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county2 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county3 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county4 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county5 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county6 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county7 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)
model_last.coe_county8 = Param(range(N_e_last), range(N_t), domain=NonNegativeReals, mutable=True)

# Constraints for individual trip chain
model_last.c1 = ConstraintList()
for e in range(N_e_last):
    for t in range(N_t):
        model_last.c1.add(expr=model_last.y_e[e, t] <= model_last.omega[e] * model_last.x_e_UB[e, t])
        model_last.c1.add(expr=E_init * model_last.omega[e] - model_last.demand_cum[e, t] * model_last.omega[e] + 
                     sum(L[t, tt] * model_last.y_e[e, tt] * 0.25 for tt in range(N_t)) == model_last.E_e_omega[e, t])
        model_last.c1.add(expr=model_last.E_e_omega[e, t] >= 0)
        model_last.c1.add(expr=model_last.E_e_omega[e, t] <= model_last.omega[e])
    model_last.c1.add(expr=model_last.E_e_omega[e, N_t-1] >= model_last.omega[e] * E_terminal)

# Constraints for aggregate charging power capacity
model_last.c2 = ConstraintList()
for t in range(N_t):
    model_last.c2.add(sum(model_last.coe_county0[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[0])
    model_last.c2.add(sum(model_last.coe_county1[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[1])
    model_last.c2.add(sum(model_last.coe_county2[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[2])
    model_last.c2.add(sum(model_last.coe_county3[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[3])
    model_last.c2.add(sum(model_last.coe_county4[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[4])
    model_last.c2.add(sum(model_last.coe_county5[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[5])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[6])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[7])
    model_last.c2.add(sum(model_last.coe_county6[e, t] * model_last.y_e[e, t] for e in range(N_e_last)) <= model_last.ell_capacity[8])

# Objective
model_last.obj = Objective(expr=sum(model_last.q_e[e] * model_last.omega[e] for e in range(N_e_last)), sense=maximize)



In [5]:
capacity_county_ori = pd.read_csv('geo_California/LoadCapacity_bay.csv', index_col = 0)
capacity_county = np.zeros(N_n)
for county_ind, county_name in index_to_county.items():
    capacity_temp = capacity_county_ori[capacity_county_ori['NAME'] == county_name]['LoadCapacity_MW'].values[0]
    capacity_county[county_ind] = capacity_temp
capacity_county *= 1000/56.5
capacityNum = 1
capacity_scale = [1]

peneList = [0.025]
peneNum = len(peneList)

ratio_pubCharger_list = [0.4]
ratioNum = len(ratio_pubCharger_list)

In [7]:
omega_rec = {}

solver = SolverFactory('gurobi')

for i in range(peneNum):
    omega_rec[i] = {}
    for ratio_pubCharger in ratio_pubCharger_list:
        flag_group = np.zeros(groupNum+1)
        omega_rec[i][ratio_pubCharger] = np.zeros((capacityNum, N_e_total))
        x_UB_value = sparse.load_npz(f"Parameters/Bay/pubCharger/p_max_ratio={ratio_pubCharger:.1f}.npz").toarray()
        for j in range(capacityNum):
            capacity_value = capacity_county * capacity_scale[j]
            print (f"start: ratio_pubCharger={ratio_pubCharger}, scale={capacity_scale[j]}, EV_pene={peneList[i]}")
            pene_value = peneList[i]
            model.pene_para = pene_value
            start_all = time.time()
            for g in range(groupNum):
                if flag_group[g] == 1:
                    omega_rec[i][ratio_pubCharger][j, g * N_e: (g+1) * N_e] = 1
                    print (f"group={g}, obj=100%")
                    continue

                '''parameter seeting for each group'''
                for n in range(N_n):
                    model.ell_capacity[n] = capacity_value[n] * groupShare_list[g]

                for e in range(N_e):
                    model.q_e[e] = q[g * N_e + e]
                    for t in range(N_t):
                        model.x_e_UB[e, t] = x_UB_value[g * N_e + e, t]
                        model.demand_cum[e, t] = demand_cum[g * N_e + e, t]
                        model.coe_county0[e, t] = pene_value * q[g * N_e + e] * boolIndex[0][g * N_e + e, t]
                        model.coe_county1[e, t] = pene_value * q[g * N_e + e] * boolIndex[1][g * N_e + e, t]
                        model.coe_county2[e, t] = pene_value * q[g * N_e + e] * boolIndex[2][g * N_e + e, t]
                        model.coe_county3[e, t] = pene_value * q[g * N_e + e] * boolIndex[3][g * N_e + e, t]
                        model.coe_county4[e, t] = pene_value * q[g * N_e + e] * boolIndex[4][g * N_e + e, t]
                        model.coe_county5[e, t] = pene_value * q[g * N_e + e] * boolIndex[5][g * N_e + e, t]
                        model.coe_county6[e, t] = pene_value * q[g * N_e + e] * boolIndex[6][g * N_e + e, t]
                        model.coe_county7[e, t] = pene_value * q[g * N_e + e] * boolIndex[7][g * N_e + e, t]
                        model.coe_county8[e, t] = pene_value * q[g * N_e + e] * boolIndex[8][g * N_e + e, t]

                '''solve the problem for each group'''
                start = time.time()
                results = solver.solve(model)

                if str(results['Solver'][0]['Termination condition']) == 'optimal':
                    omega_temp = np.array(model.omega[:]())
                    omega_rec[i][ratio_pubCharger][j, g * N_e: (g+1) * N_e] = omega_temp
                    print (f"group={g}, obj={omega_temp.mean()*100:.2f}%")
                    if omega_temp.mean() == 1:
                        flag_group[g] = 1
                else:
                    print (f"group={g}, infeasible")


            '''solve the problem for the last group'''
            if flag_group[groupNum] == 1:
                omega_rec[i][ratio_pubCharger][j, groupNum * N_e: N_e_total] = 1
                print (f"group={g}, percentage={100}%, time={time.time()-start_all}")
                continue

            model_last.pene_para = pene_value
            for n in range(N_n):
                model_last.ell_capacity[n] = capacity_value[n] * groupShare_list[groupNum]

            for e in range(N_e_last):
                model_last.q_e[e] = q[groupNum * N_e + e]
                for t in range(N_t):
                    model_last.x_e_UB[e, t] = x_UB_value[groupNum * N_e + e, t]
                    model_last.demand_cum[e, t] = demand_cum[groupNum * N_e + e, t]
                    model_last.coe_county0[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[0][groupNum * N_e + e, t]
                    model_last.coe_county1[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[1][groupNum * N_e + e, t]
                    model_last.coe_county2[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[2][groupNum * N_e + e, t]
                    model_last.coe_county3[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[3][groupNum * N_e + e, t]
                    model_last.coe_county4[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[4][groupNum * N_e + e, t]
                    model_last.coe_county5[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[5][groupNum * N_e + e, t]
                    model_last.coe_county6[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[6][groupNum * N_e + e, t]
                    model_last.coe_county7[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[7][groupNum * N_e + e, t]
                    model_last.coe_county8[e, t] = pene_value * q[groupNum * N_e + e] * boolIndex[8][groupNum * N_e + e, t]

            results_last = solver.solve(model_last)
            if str(results_last['Solver'][0]['Termination condition']) == 'optimal':
                omega_temp = np.array(model_last.omega[:]())
                omega_rec[i][ratio_pubCharger][j, groupNum * N_e: N_e_total] = omega_temp
                print (f"group={g}, percentage={omega_rec[i][ratio_pubCharger][j].mean()*100:.2f}%, time={time.time()-start_all}")
                if omega_temp.mean() == 1:
                    flag_group[groupNum] = 1
            else:
                print (f"group={g}, infeasible")

            # break # only solve one scale
        break # only solve one ratio
    break # only solve one pene
                

start: ratio_pubCharger=0.4, scale=0.5, EV_pene=0.025
group=0, obj=100.00%
group=1, obj=100.00%
group=2, obj=100.00%
group=3, obj=100.00%
group=4, obj=100.00%
group=5, obj=100.00%
group=6, obj=100.00%
group=7, obj=100.00%
group=8, obj=100.00%
group=9, obj=96.50%
group=10, obj=91.20%
group=11, obj=91.10%
group=12, obj=89.90%
group=13, obj=88.40%
group=14, obj=90.50%
group=15, obj=92.50%
group=16, obj=90.00%
group=17, obj=87.90%
group=18, obj=90.00%
group=18, percentage=95.08%, time=229.0598337650299
start: ratio_pubCharger=0.4, scale=1.0, EV_pene=0.025
group=0, obj=100%
group=1, obj=100%
group=2, obj=100%
group=3, obj=100%
group=4, obj=100%
group=5, obj=100%
group=6, obj=100%
group=7, obj=100%
group=8, obj=100%
group=9, obj=96.50%
group=10, obj=91.20%
group=11, obj=91.10%
group=12, obj=89.90%
group=13, obj=88.40%
group=14, obj=90.50%
group=15, obj=92.50%
group=16, obj=90.00%
group=17, obj=87.90%
group=18, obj=90.00%
group=18, percentage=95.08%, time=115.28720188140869


In [9]:
import os
filePath = 'Results/Bay/omega_ratio'

# for i in range(peneNum):
for i in [0]:
    for ratio_pubCharger in ratio_pubCharger_list:
        omega_sparse = sparse.csr_matrix(omega_rec[i][ratio_pubCharger])
        sparse.save_npz(f"{filePath}/omega_pene={peneList[i]:.3f}_ratio={ratio_pubCharger:.1f}.npz", omega_sparse)
        print (f"pene={peneList[i]:.3f}, ratio={ratio_pubCharger:.1f}, saved")
        break


pene=0.025, ratio=0.4, saved
